In [ ]:
crit_value = c(0.69,0.77,0.85) #### AUCROC values for effect sizes
library(cutpointr)
library(dplyr)
library(tidyr)
library(stringr)

info3.kb_area_sim = function (ref, test, prevalence = NULL){
  ###########################################
  #############Rule-in Function##############
  ##########################################
  
  pin = function(se,fn,lrp,lrn){
    
    pi = exp(se*log(lrp)+fn*log(lrn))
    
    return(pi)
    
  }
  
  ###########################################
  #############Rule-out Function##############
  ##########################################
  
  pout = function(sp,fp,lrp,lrn){
    
    po = exp(fp*log(1/lrp)+sp*log(1/lrn))
    
    return(po)
    
  }
  
  ###########################################
  #############Joint Entropy Function##############
  ##########################################
  je = function(tpcm,tncm,fpcm,fncm){
    
    joint_entropy = -(tpcm*log2(tpcm))-(tncm*log2(tncm))-(fncm*log2(fncm))-(fpcm*log2(fpcm))
    
    return(joint_entropy)
    
  }
  
  
  df = data.frame(ref, test)####The data set that includes the continous biomarker-a.k.a test- and status-a.k.a ref-
  
  
  if (is.null(prevalence)) 
    
    prevalence = sum(df$ref == 1)/length(df$ref)###specifiying prevelance
  
  tab = as.matrix(table(df$test, df$ref))#####the counts of status at each point of test value
  
  totpos = sum(tab[, 2])#######the total number of  positive's from the status
  
  totneg = sum(tab[, 1])#######the total number of negative's from the status
  
  ###########Create the data frame from the previous the counts of status at each point of test value
  
  rdf = data.frame(thresholds = unique(sort(test)), 
                   
                   d0 = tab[, "0"], d1 = tab[, "1"], row.names = 1:nrow(tab))
  
  ###################################################################
  
  rdf$tot = rowSums(tab)####the counts of observations in the each point of the data
  
  #####Calculte the TP, FP, TN, FN, and its rates from the below
  
  rdf$TP = unname(rev(cumsum(rev(tab[, 2]))))
  
  rdf$FP = unname(rev(cumsum(rev(tab[, 1]))))
  
  rdf$TN = totneg - rdf$FP
  
  rdf$FN = totpos - rdf$TP
  
  rdf$tpr = rdf$TP/totpos
  
  rdf$tnr = rdf$TN/totneg
  
  rdf$fpr = rdf$FP/totneg
  
  rdf$fnr = rdf$FN/totpos
  
  rdf$sp = 1 - rdf$fpr
  
  rdf$se = rdf$tpr
  
  #############The proportion of TP,TN,FN,FP to all sample in order to calculate joint entropy
  
  rdf$tpcm = rdf$TP/nrow(df)
  
  rdf$fpcm = rdf$FP/nrow(df)
  
  rdf$tncm = rdf$TN/nrow(df)
  
  rdf$fncm = rdf$FN/nrow(df)
  
  ############
  
  rdf$preodds = prevalence/(1 - prevalence)####preodds
  
  rdf$neg.lr = (1 - rdf$se)/rdf$sp######Negative likelihood ratio
  
  rdf$pos.lr = rdf$se/(1 - rdf$sp)#########Positive likelihood ratio
  
  overlapping1 = min(df[which(df$ref==1),2])
  
  overlapping2 = max(df[which(df$ref==0),2])
  
  rdf$pin = pin(se=rdf$se,fn=rdf$fnr,lrp=rdf$pos.lr,lrn=rdf$neg.lr)####Rule-in
  
  rdf$pout = pout(sp=rdf$sp,fp=rdf$fpr,lrp=rdf$pos.lr,lrn=rdf$neg.lr)######Rule-out
  
  ########Restrict the results for the overlapped area of the two distribution
  
  rdf_reduced = rdf[which(overlapping1 <= rdf$thresholds & rdf$thresholds <= overlapping2),]
  
  kk = rdf[rdf$pin!="Inf",]######Eliminate infinity
  
  u_pin = kk[which(kk$pin==max(kk$pin,na.rm = T)),1]###The inital maximum point to search grey zone
  
  dd = rdf[rdf$pout!="Inf",]######Eliminate infinity
  
  l_pot = dd[which(dd$pout==max(dd$pout,na.rm = T)),1]###The inital minimum point to search grey zone
  
  rdf_reduced$je = je(tpcm=rdf_reduced$tpcm, tncm=rdf_reduced$tncm, fpcm=rdf_reduced$fpcm,fncm=rdf_reduced$fncm)###Joint entropy
  
  ###Cut of value basd on Youden index kernel density
  
  df_reduced = df[which(overlapping1 <= df$test & df$test <= overlapping2),]
  
  opt1 = cutpointr(df_reduced, x = test , class=ref,method = oc_youden_kernel,pos_class=1)
  
  cut = opt1$optimal_cutpoint
  
  ####Restrict the area between cut-of and initial maximum value
  
  up = subset(rdf_reduced, thresholds>cut & thresholds<overlapping2)
  
  ####Restrict the area between cut-of and initial minimum value
  
  low = subset(rdf_reduced, thresholds>overlapping1 & thresholds<cut)
  
  low = low[,c(1,3,24)]
  
  up  = up[,c(1,3,24)]
  
  colnames(low) = c("LL","Status","JE_low")
  colnames(up) = c("UL","Status1","JE_up")
  
  
  tt = crossing(low,up, .name_repair = "unique")
  
  tt = data.frame(tt)
  
  aucs=NULL
  
  for (i in 1:nrow(tt)){
    data$gr<-ifelse(data$X<tt[i,1],0, 
                    ifelse (data$X>tt[i,4],1,2))
    table = table(data$gr,data$y)
    library(caret)
    cm = confusionMatrix(table[c(1,2),],positive="1")
    accuracy = cm$overall[1]
    sens = cm$byClass [1]
    spef = cm$byClass [2]
    X = c(sens)
    Y = 1-c(spef)
    names(X) = NULL
    names(Y) = NULL
    roc_df = as.data.frame(matrix(c(0,0,X,Y,1,1),nrow=3,byrow=T))
    X1 = roc_df$V1
    Y1 = roc_df$V2
    require(pracma)
    AUC= trapz(x=Y1,y=X1)
    aucs[i] = AUC
    
  }
  tt$aucs = aucs
  
  pm_Res= tt[tt$aucs>=crit_value[1]-0.01 & tt$aucs<=crit_value[1]+0.01,]##crit_value[1] will cahnge based on scenario
  
  pm_Res$dif =  pm_Res$UL - pm_Res$LL
  
  pm_Res$je = pm_Res$JE_low+pm_Res$JE_up
  
  bounds = pm_Res[which.max(pm_Res$je),c(1,4)]
  
  #####Find the (maximum value of joint entropy between cutof and initial maximum value
  utpos = bounds[2]
  
  
  #####Find the maximum value of joint entropy between cutof and initial minimum value
  
  ltpos = bounds[1]
  
  
  
  return(list(table = rdf_reduced,table1=rdf, thresholds = c(overlapping2,overlapping1,cut,ltpos,utpos)))
}

